# LSTM Model Training and Saving for Tkinter GUI

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import joblib
import os

In [ ]:
# Load dataset
df = pd.read_csv("realistic_threshold_noise_flow_data.csv")
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.set_index('timestamp')

# Select features
features = ['flow_rate', 'pressure', 'temperature_C']
data = df[features].copy()

In [ ]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# Save scaler
os.makedirs("models", exist_ok=True)
joblib.dump(scaler, "models/lstm_scaler.gz")

In [ ]:
def create_sequences(data, seq_length=24):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length][0])  # Predict flow_rate
    return np.array(X), np.array(y)

seq_length = 24
X, y = create_sequences(scaled_data, seq_length)

In [ ]:
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [ ]:
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(seq_length, len(features))),
    LSTM(50),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, callbacks=[early_stopping])

In [ ]:
model.save("models/lstm_model.h5")
print("✅ Model and scaler saved in 'models' directory.")